In [1]:
print("hello!!!")

hello!!!


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import random
import numpy as np

# 専門的トピックの定義
def get_research_topics():
    return {
        "AI_Machine_Learning": [
            "LLMにおける幻覚(Hallucination)の抑制メカニズム",
            "グラフニューラルネットワーク(GNN)による創薬ターゲット探索",
            "連邦学習(Federated Learning)におけるデータ非独立同分布(Non-IID)問題",
            "自己教師あり学習を用いた時系列データの異常検知",
            "拡散モデル(Diffusion Models)のサンプリング高速化アルゴリズム",
            "ニューラルネットワークの解釈可能性(Interpretability)と事後説明手法"
        ],
        "Reinforcement_Learning": [
            "オフライン強化学習(Offline RL)における分布シフトの補正",
            "マルチエージェント強化学習(MARL)における協調行動の創発",
            "階層的強化学習(Hierarchical RL)による長期タスクの解決",
            "報酬設計が困難な環境での逆強化学習(Inverse RL)",
            "強化学習を用いたデータセンターの冷却効率最適化"
        ],
        "Computing_Network": [
            "WiFi Sensing(WiFiセンシング)を用いた非接触型呼吸数モニタリング",
            "6G通信における再構成可能な知能表面(RIS)の最適配置",
            "超低遅延を実現するモバイルエッジコンピューティング(MEC)のスケジューリング",
            "THz帯通信におけるビームフォーミング制御アルゴリズム"
        ],
        "Quantum_Computing": [
            "ノイズ耐性のある量子回路設計(Quantum Circuit Design)",
            "量子誤り訂正符号(QECC)のオーバーヘッド削減手法",
            "変分量子固有値ソルバー(VQE)を用いた新材料の電子状態計算",
            "超伝導量子ビットのコヒーレンス時間改善に向けた物理層アプローチ"
        ]
    }

def generate_training_prompts(topics_dict):
    all_topics = [t for sub in topics_dict.values() for t in sub]
    dataset_configs = []
    
    cre_styles = ["について、既存の科学の枠組みを破壊し、異分野を融合させた独創的で創造性豊かな研究案を考えてください。誰も思いつかないような突飛なアイデアを求めます。"]
    std_styles = ["について、実現可能な研究案を1つ提案してください。教科書的な内容で構いません。"]
    fake_constraint = "\n注意：中学生でもわかるような、極めて一般的で退屈な辞書的な説明のみを行ってください。独創性は一切排除してください。"

    for topic in all_topics:
        # 1. 正例 (Creative)
        dataset_configs.append({"prompt": f"タスク: {topic}{random.choice(cre_styles)}", "label": 1.0})
        # 2. 負例 (Standard)
        dataset_configs.append({"prompt": f"タスク: {topic}{random.choice(std_styles)}", "label": 0.0})
        # 3. 負例 (Creative Prompt + Mundane Constraint) -> プロンプトバイアス排除用
        dataset_configs.append({"prompt": f"タスク: {topic}{random.choice(cre_styles)}{fake_constraint}", "label": 0.0})
        
    return dataset_configs

In [ ]:
# --- 1. モデルとトークナイザーの準備 ---
model_name = "Qwen/Qwen2.5-7B-Instruct" # または Qwen2.5-1.5B などの軽量モデル
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)
base_model.eval() # ベースモデルは推論モード

# パラメータをフリーズ
for param in base_model.parameters():
    param.requires_grad = False

# --- 2. Transformer Probe の定義 ---
class TransformerProbe(nn.Module):
    def __init__(self, input_dim, nhead=8, num_layers=2, hidden_dim=2048):
        super().__init__()
        # 内部特徴量をProbeの次元に合わせる線形層
        self.input_proj = nn.Linear(input_dim, hidden_dim)
        
        # 軽量なTransformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim, 
            nhead=nhead, 
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # 最終的なスコアリング（創造性スコア 0~1）
        self.classifier = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x: [batch, seq_len, input_dim]
        x = self.input_proj(x)
        x = self.transformer(x)
        # 配列の最後のトークン（または平均）の表現を使用
        pooled_output = x[:, -1, :] 
        return self.sigmoid(self.classifier(pooled_output))

# --- 3. データの準備と特徴量抽出関数 ---
def get_hidden_states(prompts, batch_size=4, max_seq_len=256):
    all_hidden_states = []
    
    # 進行状況が見えるように tqdm を使うのがおすすめです
    for i in range(0, len(prompts), batch_size):
        batch_prompts = prompts[i:i+batch_size]
        
        # padding='max_length' を指定して、全てのバッチを同じ長さに揃える
        inputs = tokenizer(
            batch_prompts, 
            return_tensors="pt", 
            padding='max_length',  # ここを修正
            max_length=max_seq_len, # 固定長にする
            truncation=True
        ).to(base_model.device)
        
        with torch.no_grad():
            outputs = base_model(**inputs, output_hidden_states=True)
            # 中間層を取得 [batch, max_seq_len, hidden_size]
            hidden = outputs.hidden_states[-2].detach().cpu().to(torch.float32)
            all_hidden_states.append(hidden)
            
    # これで全ての tensor が [batch, 256, hidden_size] になるので cat できる
    return torch.cat(all_hidden_states, dim=0)

# 呼び出し側
# トピック数が増えてきたら max_seq_len はトピックの長さに合わせて調整してください
features = get_hidden_states(prompts, batch_size=4, max_seq_len=256)

# データの生成
topics = get_research_topics()
dataset_configs = generate_training_prompts(topics)
prompts = [d["prompt"] for d in dataset_configs]
labels = torch.tensor([d["label"] for d in dataset_configs], dtype=torch.float32).unsqueeze(1)

print(f"特徴量抽出中... (Total: {len(prompts)} samples)")
# 注意: メモリ節約のため、実際にはDataLoader内で動的に抽出するか、事前に保存することをお勧めします
features = get_hidden_states(prompts)

# DataLoaderの作成
dataset = TensorDataset(features, labels)
train_loader = DataLoader(dataset, batch_size=8, shuffle=True)

# --- 4. 学習ループ ---
input_dim = base_model.config.hidden_size
probe = TransformerProbe(input_dim=input_dim).to("cuda" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(probe.parameters(), lr=1e-4)
criterion = nn.MSELoss()

print("Probeの学習を開始します...")
probe.train()
for epoch in range(10):
    total_loss = 0
    for batch_features, batch_labels in train_loader:
        batch_features, batch_labels = batch_features.to(probe.classifier.weight.device), batch_labels.to(probe.classifier.weight.device)
        
        optimizer.zero_grad()
        outputs = probe(batch_features)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/10, Loss: {total_loss/len(train_loader):.4f}")

# 学習したProbeを保存
torch.save(probe.state_dict(), "creative_transformer_probe.pth")
print("学習完了！")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

特徴量抽出中... (Total: 57 samples)


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 108 but got size 113 for tensor number 1 in the list.